In [9]:
import os
import csv
from typing import Any, Tuple, List

import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2

In [2]:
class MyDataset(Dataset):
    def __init__(self, path_to_annot_file: str, transform: Any=None, train: bool=False, test: bool=False, valid: bool=False) -> None:
        self.path_to_annot_file = path_to_annot_file
        self.data = self.get_data(path_to_annot_file, train, test, valid)
        self.transform = transform

    def get_data(self, path_to_annot_file: str, train: bool=False, test: bool=False, valid: bool=False) -> pd.DataFrame:
        data = pd.read_csv(path_to_annot_file, usecols = (0, 2), names = ('path_to_image', 'label'))
        data = data.sample(frac=1).reset_index(drop=True)
        data['label'] = data['label'].apply(lambda label: 0 if label == 'rose' else 1)
        train_size = int(0.8 * data.shape[0])
        test_valid_size = int(0.1 * data.shape[0])
        if train == True:
            data = data.iloc[:train_size, :]
        elif test == True:
            data = data.iloc[train_size:train_size + test_valid_size, :]
        elif valid == True:
            data = data.iloc[train_size + test_valid_size:, :]
        return data

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index: int) -> Tuple[torch.tensor, int]:
        path_to_image = self.data.iloc[index, 0]
        image = cv2.cvtColor(cv2.imread(path_to_image), cv2.COLOR_BGR2RGB)
        label = self.data.iloc[index, 1]
        if self.transform:
            image = self.transform(image)
        return image, label

In [3]:
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                                    torchvision.transforms.Resize((224, 224)),
                                                    torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                                    torchvision.transforms.RandomHorizontalFlip(p=0.5)])

In [8]:
train_data = MyDataset(path_to_annot_file='Annotasion.csv', transform=transforms, train = True)
test_data = MyDataset(path_to_annot_file='Annotasion.csv', transform=transforms, test = True)
valid_data = MyDataset(path_to_annot_file='Annotasion.csv', transform=transforms, valid = True)
len(train_data), len(test_data), len(valid_data)

(1600, 200, 201)

                                 absPath className
0     C:\pp var 4\dataset\tulip\0000.jpg     tulip
1      C:\pp var 4\dataset\rose\0000.jpg      rose
4     C:\pp var 4\dataset\tulip\0002.jpg     tulip
5      C:\pp var 4\dataset\rose\0002.jpg      rose
6     C:\pp var 4\dataset\tulip\0003.jpg     tulip
...                                  ...       ...
1995   C:\pp var 4\dataset\rose\0997.jpg      rose
1996  C:\pp var 4\dataset\tulip\0998.jpg     tulip
1997   C:\pp var 4\dataset\rose\0998.jpg      rose
1998  C:\pp var 4\dataset\tulip\0999.jpg     tulip
1999   C:\pp var 4\dataset\rose\0999.jpg      rose

[1586 rows x 2 columns]
                                 absPath className
2     C:\pp var 4\dataset\tulip\0001.jpg     tulip
3      C:\pp var 4\dataset\rose\0001.jpg      rose
20    C:\pp var 4\dataset\tulip\0010.jpg     tulip
22    C:\pp var 4\dataset\tulip\0011.jpg     tulip
38    C:\pp var 4\dataset\tulip\0019.jpg     tulip
...                                  ...       ...
1931  